In [1]:
import os
import argparse
import time

import matplotlib as mpl
mpl.use('TkAgg') # sets the backend for matplotlib
import matplotlib.pyplot as plt

from utils.redraw_obstacles import redraw_obstacles, set_non_blocking
from utils.create_draw_params import create_draw_params
from commonroad.common.file_reader import CommonRoadFileReader
from commonroad.visualization.draw_dispatch_cr import draw_object

import numpy as np

In [2]:
file_path = '/home/rong/VAE-Motion-Planning/scenarios/commonroad_data/highway-ramp.cr.xml'
scenario, _ = CommonRoadFileReader(file_path).open()

In [3]:
def show_lane(scenario):
    plt.figure(figsize=(250, 100))
    draw_params={'time_begin':0}
    handles = {}
    draw_object(scenario, handles=handles, draw_params=draw_params)
    plt.gca().autoscale()
    plt.gca().set_aspect('equal')
    plt.show()

In [4]:
show_lane(scenario)

KeyboardInterrupt: 

In [32]:
def plt_window(sce, dur, draw_params, size, pos):
    set_non_blocking()  # ensures interactive plotting is activated
    figsize = size
    fig = plt.figure(figsize=(figsize[0] / inch_in_cm, figsize[1] / inch_in_cm))
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    # ax.set_facecolor("black")
    # ax.set_cmap("binary")
    fig.add_axes(ax)
    fig.patch.set_facecolor('black')
#     plt.xlim(pos[0] - size[0]/2, pos[0] + size[0]/2)
#     plt.ylim( pos[1] -size[1]/2, pos[1] + size[1]/2)
    plt.set_cmap("plasma")
    handles = {}  # collects handles of obstacle patches, plotted by matplotlib
    plot_limits = [pos[0] - size[0]/2, pos[0] + size[0]/2, pos[1] +-size[1]/2, pos[1] + size[1]/2]
    draw_params['time_begin'] = dur[0]
    draw_object(sce, handles=handles, draw_params=draw_params, plot_limits=plot_limits)
    # plt.gray()
    fig.canvas.draw()
    for handles_i in handles:
        if not handles_i:
            handles.pop()

    data = np.zeros(( (duration[1] - duration[0],) + fig.canvas.get_width_height()[::-1]), dtype='bool')

    t1 = time.time()
    # loop where obstacle positions are updated
    for i in range(dur[0], dur[1]):
        # ...
        # change positions of obstacles
        # ...
        draw_params['time_begin'] = i
        redraw_obstacles(sce, handles=handles, figure_handle=fig, plot_limits=plot_limits, draw_params=draw_params)
        buffer = np.frombuffer(fig.canvas.renderer.buffer_rgba(), dtype=np.uint8).reshape(fig.canvas.get_width_height()[::-1] + (4,))
        buffer = buffer[:, : , 0].astype(bool)
        data[(i-duration[0]), :, : ] = buffer
        print('fps:', (i - duration[0])/(time.time()-t1))
    return data

In [6]:
size = [200, 40]
pos = [350, 135]
duration = [0, 100]
inch_in_cm = 10
draw_params = create_draw_params()

origin_data = plt_window(scenario=scenario, duration=duration, draw_params=draw_params, size=size, pos=pos)

fps: 0.0
fps: 11.281773751469295
fps: 15.243145806076464
fps: 16.225672925902714
fps: 17.809432922770878
fps: 18.631646093679205
fps: 18.983189131697454
fps: 19.68052005681586
fps: 20.367521160000194
fps: 20.819904826408575
fps: 21.157120842591635
fps: 21.095012118671075
fps: 20.582283295036735
fps: 20.87578045915578
fps: 20.66485357297702
fps: 19.718241702860645
fps: 19.39990911287926
fps: 19.15207305936073
fps: 18.99621924836866
fps: 18.702682785959382
fps: 18.160819853524597
fps: 17.987214185168252
fps: 17.18527568422864
fps: 16.916585125314153
fps: 16.443111164761735
fps: 16.634579141543426
fps: 16.75274925405097
fps: 16.89225853702922
fps: 17.073107588602703
fps: 17.269280936067858
fps: 17.388700071224658
fps: 17.500016352837985
fps: 17.734485860994003
fps: 17.839675727635033
fps: 17.92116845853633
fps: 18.131361387581602
fps: 18.2021226555544
fps: 18.36716953432922
fps: 18.4684946414981
fps: 18.560221264156212
fps: 18.65138348304905
fps: 18.770524878197016
fps: 18.63834136413527


In [7]:
def plt_data(data):
    fig = plt.figure()
    plt.gray()
    if len(data.shape) <=2:
        plt.imshow(data)
    else:
        myobj = plt.imshow(data[0, :, :])
        for row in data:
            myobj.set_data(row)
            plt.draw()
            plt.pause(0.01)

In [9]:
plt_data(origin_data)

In [10]:
from commonroad.scenario.scenario import Scenario

ego_scenario = Scenario(dt=0.1, benchmark_id=scenario.benchmark_id)
ego_scenario.lanelet_network = scenario.lanelet_network

In [105]:
import copy

ego_id = 142
# ego_obstacle = scenario.dynamic_obstacles[ego_id]
# ego_obstacle_new = copy.deepcopy(ego_obstacle)

In [106]:
obstacle = scenario.obstacle_by_id(ego_id)
scenario.remove_obstacle(scenario.obstacle_by_id(ego_id))
ego_scenario.add_objects(obstacle)
# show_lane(ego_scenario)

In [113]:
duration = [24394, 24450]
ego_data = plt_window(sce=ego_scenario, dur=duration, draw_params=draw_params, size=size, pos=pos)

fps: 0.0
fps: 13.011040280427466
fps: 16.92654195311034
fps: 19.547212370615128
fps: 21.208900620191493
fps: 22.715330491144154
fps: 24.187000285448196
fps: 24.772799989199918
fps: 25.563139089295372
fps: 26.436299690527363
fps: 27.434802914131495
fps: 28.479543115905575
fps: 29.231671854010205
fps: 29.402682728888003
fps: 29.37788693265079
fps: 29.767498197330344
fps: 30.354917457742808
fps: 30.761597426667265
fps: 31.140182120482304
fps: 31.49355975205648
fps: 31.956127128016004
fps: 31.565255721177657
fps: 30.553712963327804
fps: 29.183045281233476
fps: 28.308817545155247
fps: 27.977194036126786
fps: 28.286560551766357
fps: 28.405602951371993
fps: 28.436276045689613
fps: 28.595015124454974
fps: 28.530267167878996
fps: 27.902531269347186
fps: 27.439010543406685
fps: 27.143254507075067
fps: 26.53691127798394
fps: 26.531019289123595
fps: 26.00579380262905
fps: 25.449588285031524
fps: 25.15742406851102
fps: 25.044546745636055
fps: 24.470591255373154
fps: 24.39881222715354
fps: 24.313439

In [114]:
obsereved = plt_window(sce=scenario, dur=duration, draw_params=draw_params, size=size, pos=pos)

fps: 0.0
fps: 0.4555973089771908
fps: 0.5984766633848437
fps: 0.6369986789639371
fps: 0.6863782930351235
fps: 0.7011400874987492
fps: 0.7045617289392705
fps: 0.7157580966612468
fps: 0.732618450393719
fps: 0.7525175395285283
fps: 0.7550081321298793
fps: 0.7611387119056108
fps: 0.7660418919114516
fps: 0.7710341696156471
fps: 0.7770105288040398
fps: 0.7753484806951667
fps: 0.7788996087376304
fps: 0.7737832944809369
fps: 0.7696595000030686
fps: 0.7739537549191804
fps: 0.7769978475348019
fps: 0.7823626524294812
fps: 0.783313657259647
fps: 0.7860560494701441
fps: 0.7831016826729582
fps: 0.7837631239321627
fps: 0.7813093266185512
fps: 0.7810010003975066
fps: 0.7839530895837727
fps: 0.7804452632553617
fps: 0.7817403607953007
fps: 0.779698813434237
fps: 0.7814409761677661
fps: 0.7806335603636497
fps: 0.7801978818896899
fps: 0.7802603496327132
fps: 0.7790495037373624
fps: 0.7771069930589273
fps: 0.7745627218754065
fps: 0.7740406328392723
fps: 0.7739553059551203
fps: 0.7729083182937767
fps: 0.772

In [28]:
print(ego_scenario.dynamic_obstacles[0])

Dynamic Obstacle:

id: 111
initial state: 
position= [  4.85 133.42]
orientation= 0.5236
velocity= 30.0
time_step= 10



In [29]:
print(scenario.dynamic_obstacles[0])

Dynamic Obstacle:

id: 112
initial state: 
position= [  4.85 139.82]
orientation= 0.5236
velocity= 30.0
time_step= 34



In [19]:
plt_data(ego_data)

In [105]:
plt_data(obsereved)